# Attention is all you need

Coding a transformer from scratch

In [20]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [21]:
# Get Shakespeares work from Andrej Karpathy's website

url = 'https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt'
filepath = tf.keras.utils.get_file('shakespeare.txt', url)

with open(filepath) as f:
    shakespeare_text = f.read()

In [22]:
# Print the first few characters
print(shakespeare_text[:148])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?



In [23]:
unique_chars = sorted(set(shakespeare_text))
char_to_int = {char: idx for idx, char in enumerate(unique_chars)}

# How many number of distinct characters has the vocabulary:
tokens = len(unique_chars)
print(f'Number of tokens in vocabulary: {tokens}')

# How many characters has the dataset:
text_length = len(shakespeare_text)
print(f'Total length of text dataset: {text_length}')

Number of tokens in vocabulary: 65
Total length of text dataset: 1115394


### Embedding Layer

In [24]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Embedding(tokens, 10))

input_array = np.random.randint(tokens, size=(1, 1))
model.compile('rmsprop', 'sparse_categorical_crossentropy')

output_array = model.predict(input_array)
print(output_array.shape)

model.summary()
# (1, 1, 10)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
(1, 1, 10)


Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_5 (Embedding)         │ (1, 1, 10)             │           650 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 650 (2.54 KB)

 Trainable params: 650 (2.54 KB)

 Non-trainable params: 0 (0.00 B)

### Positional Encoding

In [25]:
class PositionalEncoding(tf.keras.layers.Layer):
    def __init__(self, max_len, embedding_dim, **kwargs):
        super(PositionalEncoding, self).__init__(**kwargs)
        self.max_len = max_len # maximum sequence length that the model can handle
        self.embedding_dim = embedding_dim
        
        # Create the positional encodings
        position = np.arange(max_len)[:, np.newaxis]
        div_term = np.exp(np.arange(0, embedding_dim, 2) * -(np.log(10000.0) / embedding_dim))
        pe = np.zeros((max_len, embedding_dim))
        pe[:, 0::2] = np.sin(position * div_term)
        pe[:, 1::2] = np.cos(position * div_term)
        
        # Add batch dimension e.g. (max_len,embedding_dim) -> (1,max_len,embedding_dim)
        self.pe = tf.constant(pe[np.newaxis, :, :], dtype=tf.float32)
    
    def call(self, inputs):
        seq_len = tf.shape(inputs)[1]  # Get sequence length from input
        return inputs + self.pe[:, :seq_len, :]

In [26]:
embedding_dim = 10  # Embedding dimension
max_len = 50  # Maximum sequence length

# Define the model
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(tokens, embedding_dim),
    PositionalEncoding(max_len, embedding_dim)
])

input_array = np.random.randint(tokens, size=(1, 10))  # Example input
output_array = model(input_array)

print(output_array.shape)  # Should be (1, 10, embedding_dim)

(1, 10, 10)


In [27]:
model.summary()

Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_6 (Embedding)         │ (1, 10, 10)            │           650 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ positional_encoding_3           │ (1, 10, 10)            │             0 │
│ (PositionalEncoding)            │                        │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 650 (2.54 KB)

 Trainable params: 650 (2.54 KB)

 Non-trainable params: 0 (0.00 B)

### Masked Attention Head

### Multi-Masked Attention Head

In [28]:
class MaskedAttentionHead(tf.keras.layers.Layer):
    def __init__(self, embedding_dim, num_heads, **kwargs):
        super(MaskedAttentionHead, self).__init__(**kwargs)
        self.num_heads = num_heads
        self.embedding_dim = embedding_dim
        assert embedding_dim % num_heads == 0, "embedding_dim must be divisible by num_heads"
        self.depth = embedding_dim // num_heads
        
        self.wq = tf.keras.layers.Dense(embedding_dim)
        self.wk = tf.keras.layers.Dense(embedding_dim)
        self.wv = tf.keras.layers.Dense(embedding_dim)
        self.dense = tf.keras.layers.Dense(embedding_dim)
    
    def split_heads(self, x, batch_size):
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.depth))
        return tf.transpose(x, perm=[0, 2, 1, 3])
    
    def call(self, inputs):
        q, k, v = inputs
        batch_size = tf.shape(q)[0]
        
        q = self.split_heads(self.wq(q), batch_size)
        k = self.split_heads(self.wk(k), batch_size)
        v = self.split_heads(self.wv(v), batch_size)
        
        matmul_qk = tf.matmul(q, k, transpose_b=True)
        dk = tf.cast(tf.shape(k)[-1], tf.float32)
        scaled_attention_logits = matmul_qk / tf.math.sqrt(dk)
        
        mask = tf.linalg.band_part(tf.ones_like(scaled_attention_logits), -1, 0)
        scaled_attention_logits += (mask * -1e9)
        
        attention_weights = tf.nn.softmax(scaled_attention_logits, axis=-1)
        output = tf.matmul(attention_weights, v)
        
        output = tf.transpose(output, perm=[0, 2, 1, 3])
        concat_attention = tf.reshape(output, (batch_size, -1, self.embedding_dim))
        
        return self.dense(concat_attention)

In [29]:
import numpy as np

pe = np.array([
    [0.1, 0.2, 0.3, 0.4],  # Position 0
    [0.5, 0.6, 0.7, 0.8],  # Position 1
    [0.9, 1.0, 1.1, 1.2]   # Position 2
])  # Shape: (3, 4)

print("Original pe shape:", pe.shape)

Original pe shape: (3, 4)


In [30]:
pe_expanded = pe[np.newaxis, :, :]
print("Expanded pe shape:", pe_expanded.shape)
print(pe_expanded)

Expanded pe shape: (1, 3, 4)
[[[0.1 0.2 0.3 0.4]
  [0.5 0.6 0.7 0.8]
  [0.9 1.  1.1 1.2]]]
